In [1]:
!pip3 install --user cupy-cuda102

     |█████                           | 60.4MB 34.5MB/s eta 0:00:10ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 425, in _error_catcher
    yield
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.6/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/usr/lib/python3.6/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/ssl.py", line 1012, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.6/ssl.py", line 874, in read
    return self._sslobj.read(len, buffer)

In [0]:
import cupy as cp
import numpy as np

Función gradiente:

In [0]:
def gfo_cp_mark(x):
    v = x[0:(x.size-2)]
    value1 = x[x.size-2]
    value2 = x[x.size-1]
    #m,n=Sigma.shape
    first_block = Sigma@v -1/2*value1*mu -1/2*value2*unos
    second_block = -1/2*(v.dot(mu)-r)
    third_block = -1/2*(v.dot(unos)-1)
    first_block_np = cp.asnumpy(first_block)
    second_block_np = cp.asnumpy(second_block)
    third_block_np = cp.asnumpy(third_block)
    #es necesario pasar los bloques a numpy para poder hacerles concatenate 
    return cp.asarray(np.concatenate((first_block_np,np.array([second_block_np]),np.array([third_block_np]))))

In [0]:
Sigma=cp.array([[1, 1, 1],
              [1, 1, 2],
              [1, 2, 1]],dtype=float)
Sigma

array([[1., 1., 1.],
       [1., 1., 2.],
       [1., 2., 1.]])

In [0]:
x = cp.array([1,1,1,1,1]) #vector de n+3
mu = cp.array([2,2,2]) #vector de n
unos = cp.array([1,1,1]) #vector de n y puros 1's
r = 2 #escalar

In [0]:
gfo_cp_mark(x)

array([ 1.5,  2.5,  2.5, -2. , -1. ])

Función Hessiana:

In [0]:
def Hfo_cp_mark(x):
    v = x[0:(x.size-2)]
    value1 = x[x.size-2]
    value2 = x[x.size-1]
    m,n=Sigma.shape
    first_block = Sigma 
    second_block = -1/2*mu
    third_block = -1/2*unos
    fs_block = cp.column_stack((first_block, second_block, third_block))
    fourth_block = -1/2*mu
    fifth_block = -1/2*unos
    sixth_block = cp.zeros((2,2))
    #es necesario pasar los bloques a numpy para poder hacerles concatenate y row_stack
    tf_block = np.column_stack(( np.row_stack((cp.asnumpy(fourth_block),cp.asnumpy(fifth_block))),cp.asnumpy(sixth_block)))
    return cp.asarray(np.row_stack((cp.asnumpy(fs_block), tf_block)))

In [0]:
Hfo_cp_mark(x)

array([[ 1. ,  1. ,  1. , -1. , -0.5],
       [ 1. ,  1. ,  2. , -1. , -0.5],
       [ 1. ,  2. ,  1. , -1. , -0.5],
       [-1. , -1. , -1. ,  0. ,  0. ],
       [-0.5, -0.5, -0.5,  0. ,  0. ]])

In [5]:
import pandas as pd
import fix_yahoo_finance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [0]:

def extraer_datos_yahoo(stocks, start='2015-01-01', end='2020-04-30'):
    '''
    Descarga la información de precios de acciones al cierre para un periodo determinado
    Inputs:
        stocks - lista de acciones a descargar, las 50 especificadas previamente
        'COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
        'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
        'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
        'UPS','7203.T','PEP','MRK','1398.HK','MA','T'
        start - fecha inicio a partir de la cual se requieren los precios de las acciones, formato 'YYYY-MM-DD'
        end - fecha final hasta donde se requieren los precios de las acciones, formato 'YYYY-MM-DD'
    Output:
        datos - dataframe con 50 columnas, una por acción, tantos renglones como días hábiles del periodo
        solicitado (1340 en el caso de las fechas default), cada celda es el precio de una acción al cierre, 
        de un día específico
    '''
    df_c = yf.download(stocks, start, end).Close
    base = df_c['AAPL'].dropna().to_frame()
    for i in range(0,50):
        base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
    base = base.drop(columns=['AAPL_caller'])
    base = base.rename(columns={"AAPL_other": "AAPL"})
    base = base.fillna(method='ffill')
    base = base.fillna(method='bfill')
    return base

In [9]:
datos = extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [0]:
def calcular_rendimiento_vector(x):
  """
  Función para calcular el rendimiento esperado

  params:
      x     vector de precios
  
  return:
      r_est rendimiento esperado diario
  """

  # Definimos precios iniciales y finales como arreglo alojado en la gpu
  x_o = cp.asarray(x)
  x_f = x_o[1:]

  # Calculamos los rendimientos diarios
  r = cp.log(x_f/x_o[:-1])

  return r

In [0]:
def calcular_rendimiento(X):
  """
  Función para calcular el rendimiento esperado para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
             n el número de acciones
  
  return:
      r_est rvector de rendimientos esperados
  """
  m,n = X.shape
  r_est = cp.zeros(n)
  X = cp.asarray(X)

  for i in range(n):
    r_est[i] = calcular_rendimiento_vector(X[:,i]).mean()

  return 264*r_est

In [0]:
def calcular_varianza(X):

  """
  Función para calcular el la matriz de varianzas y covarianzas para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
               n el número de acciones
  
  return:
      S  matriz de varianzas y covarianzas
  """
  m,n=X.shape
  X = cp.asarray(X)

  X_m = cp.zeros((m-1,n))

  for i in range(n):
    X_m[:,i] = calcular_rendimiento_vector(X[:,i]) - calcular_rendimiento_vector(X[:,i]).mean()

  S = (cp.transpose(X_m)@X_m)/(m-2)

  return S

In [0]:
Sigma = calcular_varianza(datos)
Sigma.shape

In [0]:
mu = calcular_rendimiento(datos)

In [18]:
mu = cp.array(mu)
mu.shape

(50,)

In [25]:
r=max(mu)
r=0.40221088
r

0.40221088

In [27]:
unos = cp.ones(50)
unos.shape

(50,)

In [29]:
x = cp.ones(52)
x.shape

(52,)

In [30]:
gfo_cp_mark(x)

array([ -0.56489031,  -0.51094653,  -0.62568983,  -0.48869763,
        -0.48725189,  -0.48645027,  -0.58909593,  -0.52489344,
        -0.58032885,  -0.69596113,  -0.56266628,  -0.50822487,
        -0.51901312,  -0.56150438,  -0.44357667,  -0.47575399,
        -0.57480599,  -0.58346318,  -0.58707119,  -0.56936279,
        -0.53612875,  -0.53183594,  -0.53730344,  -0.50715988,
        -0.53145131,  -0.56398567,  -0.6112259 ,  -0.59477978,
        -0.52978029,  -0.62486999,  -0.53381256,  -0.52965459,
        -0.51519646,  -0.52165578,  -0.56635966,  -0.44782214,
        -0.61232527,  -0.5214163 ,  -0.52280191,  -0.48755955,
        -0.48839179,  -0.59750302,  -0.52535259,  -0.48178293,
        -0.59311914,  -0.52973298,  -0.52731279,  -0.51792737,
        -0.53262563,  -0.4281568 ,  -2.02592154, -24.5       ])

In [31]:
Hfo_cp_mark(x)

array([[ 2.54562313e-04,  7.81068998e-05,  8.97849434e-05, ...,
         1.78522132e-05, -6.62714658e-02, -5.00000000e-01],
       [ 7.81068998e-05,  3.88083420e-04,  1.95027419e-04, ...,
         5.17689344e-05, -1.39147755e-02, -5.00000000e-01],
       [ 8.97849434e-05,  1.95027419e-04,  3.30223638e-04, ...,
         5.41562303e-05, -1.28891726e-01, -5.00000000e-01],
       ...,
       [ 1.78522132e-05,  5.17689344e-05,  5.41562303e-05, ...,
         2.47012209e-04,  6.61363036e-02, -5.00000000e-01],
       [-6.62714658e-02, -1.39147755e-02, -1.28891726e-01, ...,
         6.61363036e-02,  0.00000000e+00,  0.00000000e+00],
       [-5.00000000e-01, -5.00000000e-01, -5.00000000e-01, ...,
        -5.00000000e-01,  0.00000000e+00,  0.00000000e+00]])